# Evolver Loop 12 Analysis

## Key Questions:
1. Does the C++ optimizer output have overlaps?
2. Can we run longer C++ optimization to get more improvements?
3. What's the gap analysis showing?

In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
import json

getcontext().prec = 30
SCALE_FACTOR = Decimal('1e18')

print('Setup complete')

In [ ]:
# Tree shape vertices
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def create_tree_polygon(x, y, angle):
    x, y, angle = float(x), float(y), float(angle)
    coords = list(zip(TX, TY))
    poly = Polygon(coords)
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

def create_high_precision_tree(x, y, angle):
    x = Decimal(str(x))
    y = Decimal(str(y))
    angle = Decimal(str(angle))
    sf = SCALE_FACTOR
    vertices = [
        (float(Decimal('0.0') * sf), float(Decimal('0.8') * sf)),
        (float(Decimal('0.125') * sf), float(Decimal('0.5') * sf)),
        (float(Decimal('0.0625') * sf), float(Decimal('0.5') * sf)),
        (float(Decimal('0.2') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('0.1') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('0.35') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('0.075') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('0.075') * sf), float(Decimal('-0.2') * sf)),
        (float(Decimal('-0.075') * sf), float(Decimal('-0.2') * sf)),
        (float(Decimal('-0.075') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('-0.35') * sf), float(Decimal('0.0') * sf)),
        (float(Decimal('-0.1') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('-0.2') * sf), float(Decimal('0.25') * sf)),
        (float(Decimal('-0.0625') * sf), float(Decimal('0.5') * sf)),
        (float(Decimal('-0.125') * sf), float(Decimal('0.5') * sf)),
    ]
    poly = Polygon(vertices)
    poly = affinity.rotate(poly, float(angle), origin=(0, 0))
    poly = affinity.translate(poly, xoff=float(x * sf), yoff=float(y * sf))
    return poly

def validate_no_overlap_strict(trees_data):
    if len(trees_data) <= 1:
        return True, []
    polygons = [create_high_precision_tree(t['x'], t['y'], t['deg']) for t in trees_data]
    overlaps = []
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                overlaps.append((i, j))
    return len(overlaps) == 0, overlaps

def get_bbox_side(trees):
    if len(trees) == 0:
        return 0
    polygons = [create_tree_polygon(t['x'], t['y'], t['deg']) for t in trees]
    union = unary_union(polygons)
    bounds = union.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def get_score(trees, n):
    side = get_bbox_side(trees)
    return (side ** 2) / n

print('Core functions defined')

In [ ]:
# Load C++ optimizer output
print('Loading C++ optimizer output...')
cpp_df = pd.read_csv('/home/code/submission_v18.csv')
cpp_df['N'] = cpp_df['id'].astype(str).str.split('_').str[0].astype(int)

cpp_trees = {}
cpp_scores = {}

for n, g in cpp_df.groupby('N'):
    trees = []
    for _, row in g.iterrows():
        x = str(row['x']).replace('s', '')
        y = str(row['y']).replace('s', '')
        deg = str(row['deg']).replace('s', '')
        trees.append({'x': x, 'y': y, 'deg': deg})
    cpp_trees[n] = trees
    cpp_scores[n] = get_score(trees, n)

cpp_total = sum(cpp_scores.values())
print(f'C++ optimizer score: {cpp_total:.6f}')

In [ ]:
# Check for overlaps in C++ output
print('\nChecking for overlaps in C++ output...')
overlap_ns = []
for n in range(1, 201):
    valid, overlaps = validate_no_overlap_strict(cpp_trees[n])
    if not valid:
        overlap_ns.append(n)
        if len(overlap_ns) <= 10:
            print(f'  N={n}: {len(overlaps)} overlapping pairs')

print(f'\nTotal N values with overlaps: {len(overlap_ns)}')
if overlap_ns:
    print(f'Overlap N values: {overlap_ns[:20]}...')

In [ ]:
# Load baseline (exp_012) for comparison
print('\nLoading baseline (exp_012)...')
baseline_df = pd.read_csv('/home/submission/submission.csv')
baseline_df['N'] = baseline_df['id'].astype(str).str.split('_').str[0].astype(int)

baseline_trees = {}
baseline_scores = {}

for n, g in baseline_df.groupby('N'):
    trees = []
    for _, row in g.iterrows():
        x = str(row['x']).replace('s', '')
        y = str(row['y']).replace('s', '')
        deg = str(row['deg']).replace('s', '')
        trees.append({'x': x, 'y': y, 'deg': deg})
    baseline_trees[n] = trees
    baseline_scores[n] = get_score(trees, n)

baseline_total = sum(baseline_scores.values())
print(f'Baseline score: {baseline_total:.6f}')
print(f'C++ optimizer score: {cpp_total:.6f}')
print(f'Improvement: {baseline_total - cpp_total:.6f}')

In [ ]:
# Compare per-N scores
print('\nPer-N comparison (C++ vs baseline):')
improvements = []
for n in range(1, 201):
    diff = baseline_scores[n] - cpp_scores[n]
    if diff > 1e-9:
        improvements.append((n, diff))

print(f'N values improved by C++: {len(improvements)}')
if improvements:
    print('\nTop 20 improvements:')
    for n, diff in sorted(improvements, key=lambda x: -x[1])[:20]:
        print(f'  N={n:3d}: +{diff:.6f}')

In [ ]:
# Create ensemble: use C++ where it's better AND has no overlaps
print('\n' + '='*60)
print('CREATING ENSEMBLE')
print('='*60)

ensemble_trees = {}
ensemble_scores = {}
ensemble_sources = {}

for n in range(1, 201):
    # Check if C++ is better
    if cpp_scores[n] < baseline_scores[n] - 1e-9:
        # Check if C++ has no overlaps
        valid, _ = validate_no_overlap_strict(cpp_trees[n])
        if valid:
            ensemble_trees[n] = cpp_trees[n]
            ensemble_scores[n] = cpp_scores[n]
            ensemble_sources[n] = 'cpp'
        else:
            ensemble_trees[n] = baseline_trees[n]
            ensemble_scores[n] = baseline_scores[n]
            ensemble_sources[n] = 'baseline (cpp had overlaps)'
    else:
        ensemble_trees[n] = baseline_trees[n]
        ensemble_scores[n] = baseline_scores[n]
        ensemble_sources[n] = 'baseline'

ensemble_total = sum(ensemble_scores.values())
print(f'\nBaseline score: {baseline_total:.6f}')
print(f'C++ optimizer score: {cpp_total:.6f}')
print(f'Ensemble score: {ensemble_total:.6f}')
print(f'Improvement over baseline: {baseline_total - ensemble_total:.6f}')

# Count sources
cpp_count = sum(1 for s in ensemble_sources.values() if s == 'cpp')
baseline_count = sum(1 for s in ensemble_sources.values() if s == 'baseline')
overlap_fallback = sum(1 for s in ensemble_sources.values() if 'overlaps' in s)
print(f'\nSources: {cpp_count} from C++, {baseline_count} from baseline, {overlap_fallback} fallback due to overlaps')

In [ ]:
# Final validation
print('\n' + '='*60)
print('FINAL VALIDATION')
print('='*60)

final_overlaps = []
for n in range(1, 201):
    valid, _ = validate_no_overlap_strict(ensemble_trees[n])
    if not valid:
        final_overlaps.append(n)

if final_overlaps:
    print(f'WARNING: {len(final_overlaps)} N values have overlaps: {final_overlaps[:10]}...')
else:
    print('All N values pass strict validation!')

In [ ]:
# Save ensemble submission
print('\n' + '='*60)
print('SAVING ENSEMBLE SUBMISSION')
print('='*60)

rows = []
for n in range(1, 201):
    trees = ensemble_trees[n]
    for i, t in enumerate(trees):
        x_val = str(t['x']).replace('s', '')
        y_val = str(t['y']).replace('s', '')
        deg_val = str(t['deg']).replace('s', '')
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': f's{x_val}',
            'y': f's{y_val}',
            'deg': f's{deg_val}'
        })

submission_df = pd.DataFrame(rows)
print(f'Submission shape: {submission_df.shape}')

submission_df.to_csv('/home/code/experiments/013_cpp_ensemble/submission.csv', index=False)
submission_df.to_csv('/home/submission/submission.csv', index=False)
print('Submission saved!')

print(f'\nFinal score: {ensemble_total:.6f}')
print(f'Target: 68.879235')
print(f'Gap: {ensemble_total - 68.879235:.6f}')